In [ ]:
import pyscf_util

In [ ]:
import re

def extract_casscf_energy(file_path):
    # 定义正则表达式匹配 CASSCF energy 的行
    pattern = r"CASSCF energy\s*=\s*(-?\d+\.\d+)"
    
    try:
        # 打开文件并读取内容
        with open(file_path, 'r') as file:
            content = file.read()
        
        # 使用正则表达式查找匹配的内容
        matches = re.findall(pattern, content)
        
        # 如果找到匹配项，返回能量值列表
        if matches:
            energies = [float(energy) for energy in matches]
            return energies
        else:
            print("未找到匹配的 CASSCF energy 行")
            return None
    except FileNotFoundError:
        print(f"文件 {file_path} 未找到")
        return None

# 示例用法
file_path = "your_file.txt"  # 替换为你的文件路径
energies = extract_casscf_energy(file_path)

if energies:
    print("提取的 CASSCF 能量值：")
    for energy in energies:
        print(energy)

In [ ]:
# read file #

qz_5z = extract_casscf_energy("../casscf_out/qz_5z.out")
dz_tz = extract_casscf_energy("../casscf_out/dz_tz.out")

In [ ]:
BOND_LENGTH = [1.3, 1.4, 1.5, 1.6, 1.68, 1.8, 1.9, 2.0, 2.2, 2.5, 2.8, 3.2]
_dz = dz_tz[:12]
_tz = dz_tz[12:22]
_qz = qz_5z[:12]
_5z = qz_5z[12:24]

_dz[-3] = -2099.30703346704
_dz[-2] = -2099.3152075368
_tz.extend([-2099.31643066554, -2099.31885492042])

In [ ]:
from scipy.interpolate import CubicSpline

import matplotlib.pyplot as plt

# 使用 BOND_LENGTH 和 _dz 进行三次样条插值
cs_dz = CubicSpline(BOND_LENGTH, _dz)
cs_tz = CubicSpline(BOND_LENGTH, _tz)
cs_qz = CubicSpline(BOND_LENGTH, _qz)
cs_5z = CubicSpline(BOND_LENGTH, _5z)

# 生成更密集的点用于绘图
bond_length_dense = [i / 100 for i in range(int(min(BOND_LENGTH) * 100), int(max(BOND_LENGTH) * 100) + 1)]

# 计算插值值
dz_dense = cs_dz(bond_length_dense)
tz_dense = cs_tz(bond_length_dense)
qz_dense = cs_qz(bond_length_dense)
_5z_dense = cs_5z(bond_length_dense)

# 绘制势能曲线
plt.figure(figsize=(10, 6))
plt.plot(bond_length_dense, dz_dense, label="DZ", linestyle='--')
plt.plot(bond_length_dense, tz_dense, label="TZ", linestyle='-.')
plt.plot(bond_length_dense, qz_dense, label="QZ", linestyle=':')
plt.plot(bond_length_dense, _5z_dense, label="5Z", linestyle='-')
plt.scatter(BOND_LENGTH, _dz, color='blue', label="DZ Points")
plt.scatter(BOND_LENGTH, _tz, color='orange', label="TZ Points")
plt.scatter(BOND_LENGTH, _qz, color='green', label="QZ Points")
plt.scatter(BOND_LENGTH, _5z, color='red', label="5Z Points")

plt.xlabel("Bond Length (Å)")
plt.ylabel("Energy (Hartree)")
plt.title("Potential Energy Curve")
plt.legend()
plt.grid()
plt.show()